In [2]:
import torch
import json
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import plotly.graph_objects as go
from scipy.stats import norm
from einops import einsum

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

import haystack_utils
from haystack_utils import get_mlp_activations
from hook_utils import get_ablate_neuron_hook, save_activation
from pythia_160m_utils import get_neuron_accuracy, ablation_effect
import plotting_utils
from plotting_utils import plot_neuron_acts, color_binned_histogram
import hook_utils

%reload_ext autoreload
%autoreload 2

In [3]:
with open('data/tiny_stories_chatgpt.json', 'r') as f:
    prompts = json.load(f)

filtered_prompts = [prompt for prompt in prompts if not prompt.startswith("Once")]

In [4]:
model = HookedTransformer.from_pretrained("roneneldan/TinyStories-1M",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device="cuda")

# large_acts_df = plotting_utils.get_neuron_moments(model, prompts,
#                                                   [[i, j] for i in range(8) for j in range(256)], hook_pre=True)

Using pad_token, but it is not set yet.


Loaded pretrained model roneneldan/TinyStories-1M into HookedTransformer


### Unigrams

In [39]:
# Similar words stored near in the embed
tokens = model.to_tokens("Once upon a time", prepend_bos=False)
logits = model.W_E[tokens].squeeze(0) @ model.W_U

for i in range(4):
    values, indices = torch.topk(logits[i], 10)
    print(model.to_str_tokens(indices))

['Once', 'One', 'Yesterday', 'At', 'There', 'Today', 'On', 'Em', 'When', 'In']
[' upon', 'bie', 'ancer', 'itting', 'll', 'aj', 'packs', 'upon', 'uttering', ' haven']
[' a', ' an', ' the', ' some', ' two', ' something', ' another', ' one', ' very', ' many']
[' time', ' day', ' week', ' evening', ' morning', ' afternoon', ' night', ' Sunday', ' way', ' year']


In [13]:
# Associations
tokens = model.to_tokens("cow", prepend_bos=False)
associated_tokens = []
for string in "grass brown field moo meat milk udders animal".split(' '):
    token = model.to_tokens(string, prepend_bos=False)
    associated_tokens.append(token[0, 0])
associated_tokens = torch.stack(associated_tokens)
logits = (model.W_E[tokens].squeeze(0) @ model.W_U).squeeze(0)
mean_logit = logits.mean(0)
mean_associated_logit = logits[associated_tokens].mean(0)

print(mean_logit, mean_associated_logit)

values, indices = torch.topk(logits, 10)
print(model.to_str_tokens(indices))

tensor(-5.6169e-09, device='cuda:0') tensor(0.2188, device='cuda:0')
['cow', 'ns', 'ridge', 'wo', ' seventeen', 'eight', 'umbers', 'please', 'ucks', 'icians']


In [6]:
print(model.generate('Once'))

  0%|          | 0/10 [00:00<?, ?it/s]

Once upon a time, there was a big bear who


In [4]:
# Generate heaps of token sequences and average their cache, then find the difference with the OUAT neurons
# Ablate components until OUAT fails
# n-l AND?

_, cache = model.run_with_cache(model.to_tokens(prompts)[:, :40])

def rand_hook(value, hook):
    cache_val = cache[hook.name][:value.size(0), :value.size(1), :value.size(2)]
    mean_val = cache_val.mean(dim=0)
    broadcasted_val = mean_val.unsqueeze(0).expand_as(value)
    value = broadcasted_val
    return value

prompt = "Once"
print(model.generate(prompt, 20, temperature=0))
for layer in range(model.cfg.n_layers):
    with model.hooks([(f'blocks.{layer}.hook_attn_out', rand_hook)]):
        print(f"Attn {layer}", model.generate(prompt, 20, temperature=0))
    with model.hooks([(f'blocks.{layer}.hook_mlp_out', rand_hook)]):
        print(f"MLP {layer}", model.generate(prompt, 20, temperature=0))

# Necessary components:
# MLP0, MLP1, MLP2, MLP3, MLP4, MLP5, MLP7 (every MLP but 6)

  0%|          | 0/20 [00:00<?, ?it/s]

Once upon a time, there was a little girl named Lily. She loved to play outside in the park


  0%|          | 0/20 [00:00<?, ?it/s]

Attn 0 Once upon a time, there was a little girl named Lily. She loved to play with her toys and


  0%|          | 0/20 [00:00<?, ?it/s]

...

MLP 6 Once upon a time, upon a a a a a a a a a and princess. She and princess


  0%|          | 0/20 [00:00<?, ?it/s]

Attn 7 Once upon a time, there was a little girl named Timmy. Timmy was very sad because he


  0%|          | 0/20 [00:00<?, ?it/s]

MLP 7 Once?





















### Ablate by cosine sim - degrades around 80

In [83]:
# Figure out which neurons directly write to each vocab
# And see if we can ablate everything else

tokens = model.to_tokens("Once upon a time", prepend_bos=False)  # [1, 4]
token_dirs = model.tokens_to_residual_directions(tokens)[0]  # [4, 64]
token_dirs_reshaped = token_dirs.unsqueeze(1).unsqueeze(1)  # [4, 1, 1, 64]
W_out_reshaped = model.W_out.unsqueeze(0)  # [1, 8, 256, 64]

cosine_sim = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
result = cosine_sim(token_dirs_reshaped, W_out_reshaped)  # [4, 8, 256]

_, cache = model.run_with_cache(tokens)
acts = [cache[f'blocks.{layer}.mlp.hook_post'][0] for layer in range(model.cfg.n_layers)] # [[batch pos]]*n_layers
acts = torch.stack(acts, dim=1) # 


layer_neuron_tuples = []
for token_index in range(result.size(0)):
    values, indices = torch.topk(result[token_index].view(-1), 20, dim=-1)
    layer_indices, neuron_indices = np.unravel_index(indices.cpu().numpy(), (8, 256))
    layer_neuron_tuples.extend(zip(layer_indices.tolist(), neuron_indices.tolist()))


In [117]:
layer_neuron_dict = haystack_utils.get_neurons_by_layer(layer_neuron_tuples)

sorted_layer_neuron_tuples = []
sorted_acts = []

for layer in layer_neuron_dict.keys():
    neurons = layer_neuron_dict[layer]
    mean_acts = haystack_utils.get_mlp_activations(filtered_prompts, layer, model, context_crop_start=2, hook_pre=False, neurons=neurons, disable_tqdm=True)
    sorted_layer_neuron_tuples.extend([(layer, neuron) for neuron in neurons])
    sorted_acts.extend(mean_acts)
    assert len(sorted_layer_neuron_tuples) == len(sorted_acts)

hooks = hook_utils.get_ablate_context_neurons_hooks(sorted_layer_neuron_tuples, sorted_acts)
with model.hooks(hooks):
    print(model.generate("Once", 10, temperature=0))

  0%|          | 0/10 [00:00<?, ?it/s]

Once in a time, a little girl named Amy was


### Ablate by DLA

In [64]:
def batched_dot_product(x, y):
    return torch.vmap(torch.dot)(x, y)
    

def mlp_DLA(prompt: str, model: HookedTransformer) -> tuple[Float[Tensor, "component"], list[str]]:
    tokens = model.to_tokens(prompt)
    answers = tokens[:, 1:]
    tokens = tokens[:, :-1]
    answer_residual_directions = model.tokens_to_residual_directions(answers)  # [pos d_model]
    _, cache = model.run_with_cache(tokens)
    neurons, labels = cache.get_full_resid_decomposition(-1, expand_neurons=True, apply_ln=True, return_labels=True)

    results = []
    for i in range(4):
        results.append(batched_dot_product(neurons.reshape(4, 2179, 64)[i], answer_residual_directions.squeeze(0).unsqueeze(1).repeat(1, 2179, 1)[i]))
    return torch.stack(results), labels


def get_unspecified_neurons(model: HookedTransformer, neurons: list[tuple[int, int]]):
    layer_dict = haystack_utils.get_neurons_by_layer(neurons)
    unspecified = []
    for layer in range(model.cfg.n_layers):
        for neuron in range(model.cfg.d_mlp):
            if not neuron in layer_dict[layer]:
                unspecified.append((layer, neuron))
    return unspecified

In [77]:
attrs, labels = mlp_DLA("Once upon a time", model)
# haystack_utils.line(neuron_attrs[0].cpu().numpy(), xlabel="Correct logit", ylabel="", title="DLA per neuron in layer")

Tried to stack head results when they weren't cached. Computing head results now


In [80]:
neuron_indices = [i for i in range(len(labels)) if 'N' in labels[i]]
neuron_labels = [labels[i] for i in neuron_indices]
neuron_attrs = attrs[:, neuron_indices]

In [85]:
dla_layer_neuron_tuples = []
for token_index in range(4):
    values, indices = torch.topk(neuron_attrs[token_index], 5, dim=-1)
    layer_indices, neuron_indices = np.unravel_index(indices.cpu().numpy(), (model.cfg.n_layers, model.cfg.d_mlp))
    dla_layer_neuron_tuples.extend(zip(layer_indices.tolist(), neuron_indices.tolist()))

print(dla_layer_neuron_tuples[:5])
print(layer_neuron_tuples[:5])

[(3, 238), (0, 4), (0, 35), (0, 5), (0, 34)]
[(0, 78), (0, 22), (2, 236), (2, 125), (5, 164)]


In [86]:
layer_neuron_dict = haystack_utils.get_neurons_by_layer(dla_layer_neuron_tuples)

sorted_layer_neuron_tuples = []
sorted_acts = []

for layer in layer_neuron_dict.keys():
    neurons = layer_neuron_dict[layer]
    mean_acts = haystack_utils.get_mlp_activations(filtered_prompts, layer, model, context_crop_start=2, hook_pre=False, neurons=neurons, disable_tqdm=True)
    sorted_layer_neuron_tuples.extend([(layer, neuron) for neuron in neurons])
    sorted_acts.extend(mean_acts)
    assert len(sorted_layer_neuron_tuples) == len(sorted_acts)

hooks = hook_utils.get_ablate_context_neurons_hooks(sorted_layer_neuron_tuples, sorted_acts)
with model.hooks(hooks):
    print(model.generate("Once", 10, temperature=0))

  0%|          | 0/10 [00:00<?, ?it/s]

Once upon a time, there was a little girl named


In [89]:
dla_layer_neuron_tuples = []
for token_index in range(4):
    values, indices = torch.topk(neuron_attrs[token_index].view(-1), 5, dim=-1)
    layer_indices, neuron_indices = np.unravel_index(indices.cpu().numpy(), (8, 256))
    dla_layer_neuron_tuples.extend(zip(layer_indices.tolist(), neuron_indices.tolist()))

unspecified_neuron_tuples = get_unspecified_neurons(model, dla_layer_neuron_tuples)
layer_neuron_dict = haystack_utils.get_neurons_by_layer(unspecified_neuron_tuples)

sorted_layer_neuron_tuples = []
sorted_acts = []

for layer in layer_neuron_dict.keys():
    neurons = layer_neuron_dict[layer]
    mean_acts = haystack_utils.get_mlp_activations(filtered_prompts, layer, model, context_crop_start=2, hook_pre=False, neurons=neurons, disable_tqdm=True)
    sorted_layer_neuron_tuples.extend([(layer, neuron) for neuron in neurons])
    sorted_acts.extend(mean_acts)
    assert len(sorted_layer_neuron_tuples) == len(sorted_acts)

hooks = hook_utils.get_ablate_context_neurons_hooks(sorted_layer_neuron_tuples, sorted_acts)
with model.hooks(hooks):
    print(model.generate("Once", 10, temperature=0))

  0%|          | 0/10 [00:00<?, ?it/s]

OnceOnceOnceOnceOnceOnceOnceOnceOnceOnceOnce
